In [1]:
import pandas as pd
# import mysql.connector as mycon
import kagglehub
from kagglehub import KaggleDatasetAdapter

# This pulls in the wildfire data and reads it

In [2]:
use_cols = ['OBJECTID', 'FIRE_NAME', 'DISCOVERY_DATE', 'NWCG_GENERAL_CAUSE', 'FIRE_SIZE', 'STATE', 'FIPS_NAME']

# Download latest version
wildfire_df = kagglehub.load_dataset(handle = "behroozsohrabi/us-wildfire-records-6th-edition", path = "data.csv", 
                                   adapter = KaggleDatasetAdapter.PANDAS, 
                                   pandas_kwargs={"usecols": use_cols, "compression": "zip"})

c:\Users\Taylor\AppData\Local\Programs\Python\Python313\Lib\site-packages\kagglehub\pandas_datasets.py:91: DtypeWarning: Columns (14,38) have mixed types. Specify dtype option on import or set low_memory=False.
  result = read_function(


In [3]:
# Convert DISCOVERY_DATE to datetime object to remove records prior to year 2000

wildfire_df['DISCOVERY_DATE'] = pd.to_datetime(wildfire_df['DISCOVERY_DATE'], format = ('%m/%d/%Y'))

wildfire_df = wildfire_df[wildfire_df['DISCOVERY_DATE'].dt.year > 2000]

In [4]:
# Rename some columns for clarity

wildfire_df = wildfire_df.rename(columns = {"NWCG_GENERAL_CAUSE": "SPECIFIC_CAUSE", "FIPS_NAME": "COUNTY"})

In [5]:
# Filter states that we're keeping

states_to_keep = ['CA', 'TX', 'GA', 'FL', 'AZ']

wildfire_df.loc[~wildfire_df['STATE'].isin(states_to_keep), :] = None

wildfire_df.dropna(inplace = True)

In [6]:
# Change to int for compatiabilty with Primary Key

wildfire_df['OBJECTID'] = wildfire_df['OBJECTID'].astype(int)

# This pulls in housing data

In [7]:
bottom_tier_housing = pd.read_parquet('data/bottom_housing.parquet')
top_tier_housing = pd.read_parquet('data/top_housing.parquet')

## Remove Columns we don't use

In [8]:
remove_cols = ['RegionID', 'RegionType', 'StateName', 'Metro', 'SizeRank']

bottom_tier_housing.drop(columns = remove_cols, inplace = True)
top_tier_housing.drop(columns = remove_cols, inplace = True)

## Remove rows with states we aren't using in a way that modifies the original dataframe in-place

In [9]:
states = ['CA', 'TX', 'GA', 'FL', 'AZ']

bottom_tier_housing.loc[~bottom_tier_housing['State'].isin(states), :] = None
top_tier_housing.loc[~top_tier_housing['State'].isin(states), :] = None

bottom_tier_housing.dropna(inplace = True)
top_tier_housing.dropna(inplace = True)


## Combine the data frames and pivot long the dates and prices

In [10]:
housing_df = pd.concat([bottom_tier_housing, top_tier_housing], axis = 0, ignore_index = True)

In [11]:
housing_df = housing_df.melt(id_vars = ['RegionName', 'State', 'CountyName'], var_name = 'Date', value_name = 'Price')
housing_df['Date'] = pd.to_datetime(housing_df['Date']).dt.date

# Rent Data (load and remove columns'state values)

In [12]:
rent_index = pd.read_csv("data/Observed Rent Index by City.csv")

In [13]:
# Columns and state values to remove are same as housing sales data

rent_index.drop(columns = remove_cols, inplace = True)
rent_index.loc[~rent_index['State'].isin(states), :] = None

rent_index.dropna(inplace = True)

## Pivot long the date and price data

In [14]:
rent_index = rent_index.melt(id_vars = ['RegionName', 'State', 'CountyName'], var_name = 'Date', value_name = 'Price')
rent_index['Date'] = pd.to_datetime(housing_df['Date']).dt.date

# Create Location Table

## Pull in location info from Census

In [15]:
ca_codes = pd.read_csv('https://www2.census.gov/geo/docs/reference/codes2020/place_by_cou/st06_ca_place_by_county2020.txt', delimiter = '|')
tx_codes = pd.read_csv('https://www2.census.gov/geo/docs/reference/codes2020/place_by_cou/st48_tx_place_by_county2020.txt', delimiter = '|')
ga_codes = pd.read_csv('https://www2.census.gov/geo/docs/reference/codes2020/place_by_cou/st13_ga_place_by_county2020.txt', delimiter = '|')
fl_codes = pd.read_csv('https://www2.census.gov/geo/docs/reference/codes2020/place_by_cou/st12_fl_place_by_county2020.txt', delimiter = '|')
az_codes = pd.read_csv('https://www2.census.gov/geo/docs/reference/codes2020/place_by_cou/st04_az_place_by_county2020.txt', delimiter = '|')

## Drop Columns

In [16]:
columns_to_drop = ['PLACENS', 'TYPE', 'CLASSFP', 'FUNCSTAT']

ca_codes.drop(columns = columns_to_drop, inplace = True)
tx_codes.drop(columns = columns_to_drop, inplace = True)
ga_codes.drop(columns = columns_to_drop, inplace = True)
fl_codes.drop(columns = columns_to_drop, inplace = True)
az_codes.drop(columns = columns_to_drop, inplace = True)

In [17]:
# Combine df to make it easier to load

combined_df = pd.concat([ca_codes, tx_codes, ga_codes, fl_codes, az_codes], axis = 0, ignore_index=True)

# Load Population Data into dictionary

In [18]:
state_population_dict = {}

states = ['CA', 'TX', 'GA', 'FL', 'AZ']

for state in states:
    state_population_dict[state] = pd.read_csv(f"data/{state} City population estimates.csv")

# Sync population and location data

In [19]:
location_dict = {
    'CA': ca_codes,
    'TX': tx_codes,
    'GA': ga_codes,
    'FL': fl_codes,
    'AZ': az_codes
}

In [20]:
def sync(states_list):
    for state in states_list:
        pop_df = state_population_dict[state]
        location_df = location_dict[state]

        columns_to_rename = {'PLACE': 'PLACEFP', 'NAME': 'PLACENAME'}

        pop_df = pop_df.rename(columns = columns_to_rename)

        merged_df = pop_df.merge(
            location_df[['COUNTYFP', 'COUNTYNAME', 'PLACEFP', 'PLACENAME']],
            on = ["PLACEFP", 'PLACENAME'],
            how = 'left'
        )
        merged_df.drop(columns = ['SUMLEV', 'COUSUB', 'CONCIT', 'PRIMGEO_FLAG', 'FUNCSTAT', 'ESTIMATESBASE2010', 'COUNTY'], inplace = True)
        merged_df = merged_df.drop_duplicates()
        merged_df = merged_df.dropna()
        merged_df['COUNTYFP'] = merged_df['COUNTYFP'].astype(int)
        state_population_dict[state] = merged_df

In [21]:
sync(states)

# Map state and county codes to wildfire/housing/rent data

In [22]:
# Mapping only for CA, TX, FL, GA, and AZ

state_fips_mapping = {
    "CA": 6,  # California
    "TX": 48, # Texas
    "FL": 12, # Florida
    "GA": 13, # Georgia
    "AZ": 4   # Arizona
}

wildfire_df['state_id'] = wildfire_df['STATE'].map(state_fips_mapping)
housing_df['state_id'] = housing_df['State'].map(state_fips_mapping)
rent_index['state_id'] = rent_index['State'].map(state_fips_mapping)

In [28]:
# Realign column names to prepare for county code mapping

combined_df.rename(columns = {'COUNTYNAME': 'county_name', 'COUNTYFP': 'county_id', 'STATEFP': 'state_id'}, inplace = True)
rent_index.rename(columns = {'CountyName': 'county_name'}, inplace = True)
housing_df.rename(columns = {'CountyName': 'county_name'}, inplace = True)
wildfire_df.rename(columns = {'COUNTY': 'county_name'}, inplace = True)

In [24]:
import pandas as pd
from thefuzz import fuzz, process  # Fuzzy matching

def match_county_id(data_df, counties_df, state_col="state_id", county_col="county_name", threshold=85):
    """
    Matches county names in the dataset to official counties using fuzzy matching.

    Parameters:
    - data_df (pd.DataFrame): DataFrame containing records that need county_id assignment.
    - counties_df (pd.DataFrame): Official Census counties with state_id, county_name, and county_id.
    - state_col (str): Column name for state_id.
    - county_col (str): Column name for county_name.
    - threshold (int): Minimum similarity score to consider a match.

    Returns:
    - pd.DataFrame: Updated DataFrame with county_id assigned.
    """

    # Create a dictionary of county names for each state
    county_lookup = {}
    for state_id in counties_df[state_col].unique():
        state_counties = counties_df[counties_df[state_col] == state_id][["county_name", "county_id"]]
        county_lookup[state_id] = state_counties.set_index("county_name")["county_id"].to_dict()

    # Function to find the best fuzzy match
    def find_best_match(state_id, county_name):
        if state_id in county_lookup:
            choices = list(county_lookup[state_id].keys())  # List of county names for the state
            match, score = process.extractOne(county_name, choices, scorer=fuzz.token_sort_ratio)
            if score >= threshold:
                return county_lookup[state_id][match]  # Return matched county_id
        return None  # No match found

    # Apply fuzzy matching to each row
    data_df["county_id"] = data_df.apply(lambda row: find_best_match(row[state_col], row[county_col]), axis=1)

    return data_df


In [ ]:
rent_index = match_county_id(rent_index, combined_df)
housing_df = match_county_id(housing_df, combined_df)
wildfire_df = match_county_id(wildfire_df, combined_df)
population_df = pd.concat(state_population_dict.values(), ignore_index = True)

NameError: name 'state_population' is not defined

In [76]:
wildfire_df.columns

Index(['OBJECTID', 'FIRE_NAME', 'DISCOVERY_DATE', 'SPECIFIC_CAUSE',
       'FIRE_SIZE', 'STATE', 'county_name', 'state_id', 'county_id'],
      dtype='object')

In [77]:
# Take care of missing values in wildfire dataframe, fill in variables accordingly

def fill_missing_values(df):
    """
    Fill missing values based on the state column.
    """
    df.loc[df['STATE'] == 'FL', 'county_id'] = df.loc[df['STATE'] == 'FL', 'county_id'].fillna(86)
    df.loc[df['STATE'] == 'GA', 'county_id'] = df.loc[df['STATE'] == 'GA', 'county_id'].fillna(29)
    return df

wildfire_df = fill_missing_values(wildfire_df)


In [ ]:
# Final dataframes

rent_index
housing_df
wildfire_df
combined_df